In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import numpy as np

In [6]:
# Inicializa el navegador
driver = webdriver.Chrome(executable_path='C:\Windows\System32\chromedriver.exe')

# Maximiza la ventana del navegador (abre en pantalla completa)
driver.maximize_window()

# Abre la página web
driver.get("https://www.autohero.com/es/search/")

# Lista para almacenar los enlaces
links = []

# Simula hacer scroll y carga los enlaces en cada scroll
for _ in range(100):  # Ajusta el número de scrolls según sea necesario
    # Hace scroll hasta el final de la página
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Espera a que la página se cargue después del scroll
    time.sleep(2)
    # Extrae los enlaces después de cada scroll y agrega a la lista
    enlaces = driver.find_elements(By.CSS_SELECTOR, 'a.link___2Maxt')
    for enlace in enlaces:
        links.append(enlace.get_attribute('href'))

# Elimina duplicados de la lista de enlaces
links = list(set(links))

# Cierra el navegador
driver.quit()

In [1]:
#Crea el dataframe con los datos de cada link
datos = pd.DataFrame()
for pagina in links[650:]:
    #si hacemos muchas consultas a la página nos echa así que vamos a poner una espera
    time.sleep(2)
    url = pagina
    r = requests.get(url)
    time.sleep(2)
    html_contents = r.text
    html_soup = BeautifulSoup(html_contents, 'html.parser')
    
    time.sleep(2)
    #Estos valores siempre aparecen en cada página
    #Vamos a sacar los elementos por su clase
    tit = html_soup.select_one('.titleText___edZSG')
    try:
        nombre = tit.text
        marca = nombre.split(' ', 1)[0]
        modelo = nombre.split(' ', 1)[1]
        precio = float(html_soup.select_one('.vehiclePrice___1uUmJ').text.strip('\xa0€').replace('.',''))
        version = html_soup.select_one('.subtitleText___2wcYx').text
        listaValores = [marca, modelo, precio, version]
        listaNombres = ['Marca', 'Modelo', 'Precio', 'Versión']

        #El resto de valores pueden o no aparecer en cada página
        #primero tratamos las etiquetas grandes de características
        carac1 = html_soup.select('.listItemValue___1IWSE')
        caracteristicas1 = [i.text for i in carac1]
        nombrecarac1 = html_soup.select('.listItemTitle___2CQBv')
        nombrecaracteristicas1 = [i.text for i in nombrecarac1]
        listaValores.extend(caracteristicas1)
        listaNombres.extend(nombrecaracteristicas1)

        #ahora se extrae la información de las tablas inferiores
        carac2 =html_soup.select('.body___2uId6')
        caracteristicas2 = [i.text for i in carac2]
        listaValores.extend(caracteristicas2)
        listaVal = [listaValores]
        nombrecarac2 =html_soup.select('.itemTitle___3GH8k')
        nombrecaracteristicas2 = [i.text for i in nombrecarac2]
        listaNombres.extend(nombrecaracteristicas2)
        datos_pagina = pd.DataFrame(listaVal,columns=listaNombres)

        datos = pd.concat([datos, datos_pagina], ignore_index=True)
    except:
        datos = datos

NameError: name 'pd' is not defined

In [13]:
datos

,Marca,Modelo,Precio,Versión,Primera matriculación,Kilometraje,Carburante,Transmisión,Potencia,Tracción,...,Estándar de calidad,País de origen,Número de llaves,Coche accidentado y reparado,La última revisión se realizó el,Historial de revisiones,Tipo de IVA,ITV válida hasta,Matricula,Número de inventario
0,Skoda,Citigo,7799.0,1.0 MPI Ambition,05.07.2016,19.507 km,Gasolina,Cambio tipo manual,60 CV / 44 kW,Tracción delantera,...,Ver estándar de calidad,España,2,No,12.03.2024,Ver historial de revisiones,IVA no deducible,13.07.2024,9958JRH,FJ27714
1,Citroen,C4 Spacetourer,14999.0,1.5 Blue-HDi Feel,27.09.2019,55.307 km,Diésel,Cambio tipo manual,131 CV / 96 kW,Tracción delantera,...,Ver estándar de calidad,España,2,No,12.03.2024,Ver historial de revisiones,IVA no deducible,13.10.2025,7578LBK,EH84421
2,Opel,Corsa,8099.0,1.4 Expression,30.04.2015,59.429 km,Gasolina,Cambio tipo manual,90 CV / 66 kW,Tracción delantera,...,Ver estándar de calidad,España,2,No,29.04.2024,Ver historial de revisiones,IVA no deducible,04.05.2025,9896JFD,MT22167
3,MG,ZS,14399.0,1.5 VTi Luxury,31.10.2022,88.984 km,Gasolina,Cambio tipo manual,106 CV / 78 kW,Tracción delantera,...,Ver estándar de calidad,España,2,No,23.06.2023,Ver historial de revisiones,IVA no deducible,27.10.2026,8050MBZ,KK46355
4,Skoda,Kamiq,16899.0,1.0 TSI Ambition,16.12.2019,5993 km,Gasolina,Cambio tipo manual,116 CV / 85 kW,Tracción delantera,...,Ver estándar de calidad,España,2,No,03.10.2023,Ver historial de revisiones,IVA no deducible,16.12.2025,9421LDD,ZM23393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007,Renault,Megane,15199.0,1.2 TCe Energy GT Line,31.01.2018,56.256 km,Gasolina,Cambio tipo manual,132 CV / 97 kW,Tracción delantera,...,NaN,España,1,NaN,NaN,NaN,IVA no deducible,NaN,0051KHF,PL11771
1008,Audi,A4,20999.0,2.0 TDI Design quattro,29.09.2016,106.137 km,Diésel,Cambio tipo automático,190 CV / 140 kW,Tracción total (4x4),...,Ver estándar de calidad,España,2,No,13.12.2023,Ver historial de revisiones,IVA no deducible,21.03.2025,2742JSY,CC37637
1009,Peugeot,3008,12599.0,1.6 Blue-HDi Crossway,10.06.2015,45.063 km,Diésel,Cambio tipo manual,120 CV / 88 kW,Tracción delantera,...,Ver estándar de calidad,España,2,No,29.02.2024,Ver historial de revisiones,IVA no deducible,22.06.2025,7371JGB,KY56106
1010,Renault,Captur,21299.0,1.3 TCe Mild-Hybrid Techno,29.11.2022,2522 km,Gasolina,Cambio tipo manual,140 CV / 103 kW,Tracción delantera,...,Ver estándar de calidad,España,2,No,05.12.2023,Ver historial de revisiones,IVA no deducible,29.11.2026,3180MCN,UM92200


In [14]:
datos.to_excel('coches1.xlsx',index=False)